# Membina Ejen AI dengan Memori Berterusan menggunakan Cognee

Notebook ini menunjukkan cara membina ejen AI pintar dengan keupayaan memori yang canggih menggunakan [**cognee**](https://www.cognee.ai/) - memori AI sumber terbuka yang menggabungkan graf pengetahuan, carian semantik, dan pengurusan sesi untuk mencipta sistem AI yang sedar konteks.

## 🎯 Objektif Pembelajaran

Pada akhir tutorial ini, anda akan memahami cara untuk:
- **Membina Graf Pengetahuan Disokong oleh Embedding**: Menukar teks tidak berstruktur kepada pengetahuan yang berstruktur dan boleh ditanya
- **Melaksanakan Memori Sesi**: Mewujudkan perbualan berbilang giliran dengan pengekalan konteks secara automatik
- **Menyimpan Perbualan**: Secara opsional menyimpan interaksi penting dalam memori jangka panjang untuk rujukan masa depan
- **Bertanya Menggunakan Bahasa Semula Jadi**: Mengakses dan memanfaatkan konteks sejarah dalam perbualan baru
- **Visualisasi Memori**: Meneroka hubungan dalam graf pengetahuan ejen anda


## 🏗️ Apa Yang Akan Anda Bina

Dalam tutorial ini, kita akan mencipta **Pembantu Pengaturcaraan** dengan memori berterusan yang:

### 1. **Pembinaan Pangkalan Pengetahuan**
   - Mengambil maklumat profil dan kepakaran pembangun
   - Memproses prinsip dan amalan terbaik pengaturcaraan Python
   - Menyimpan perbualan sejarah antara pembangun dan pembantu AI

### 2. **Perbualan Sedar Sesi**
   - Mengekalkan konteks merentasi pelbagai soalan dalam sesi yang sama
   - Secara automatik menyimpan setiap pasangan soalan/jawapan untuk pengambilan yang cekap
   - Memberikan respons yang koheren dan berkonteks berdasarkan sejarah perbualan

### 3. **Memori Jangka Panjang**
   - Menyimpan perbualan penting ke dalam memori jangka panjang
   - Mengambil memori yang relevan dari pangkalan pengetahuan dan sesi lalu untuk memaklumkan interaksi baru
   - Membina pangkalan pengetahuan yang berkembang dan bertambah baik dari masa ke masa

### 4. **Pengambilan Memori Pintar**
   - Menggunakan carian semantik yang sedar graf untuk mencari maklumat yang relevan di seluruh pengetahuan yang disimpan
   - Menapis carian mengikut subkumpulan data (maklumat pembangun vs. prinsip)
   - Menggabungkan pelbagai sumber data untuk memberikan jawapan yang komprehensif


## 📋 Prasyarat & Persediaan

### Keperluan Sistem

Sebelum memulakan, pastikan anda mempunyai:

1. **Persekitaran Python**
   - Python 3.9 atau lebih tinggi
   - Persekitaran maya (disyorkan)
   
2. **Redis Cache** (Diperlukan untuk Pengurusan Sesi)
   - Redis Tempatan: `docker run -d -p 6379:6379 redis`
   - Atau gunakan perkhidmatan Redis yang diuruskan
   
3. **Akses API LLM**
   - Kunci API OpenAI atau penyedia lain (lihat [dokumentasi](https://docs.cognee.ai/setup-configuration/llm-providers))

4. **Konfigurasi Pangkalan Data**
   - Tiada konfigurasi diperlukan secara lalai. Cognee menggunakan pangkalan data berasaskan fail (LanceDB dan Kuzu)
   - Secara opsyenal, anda boleh menyediakan Azure AI Search sebagai stor vektor (lihat [dokumentasi](https://github.com/topoteretes/cognee-community/tree/main/packages/vector/azureaisearch))

### Konfigurasi Persekitaran

Cipta fail `.env` dalam direktori projek anda dengan pembolehubah berikut:

```ini
# LLM Configuration (Required)
LLM_API_KEY=your-openai-api-key-here

# Cache Configuration (Required for Sessions)
CACHING=true  # Must be enabled for session history

```


## 🏛️ Memahami Seni Bina Memori Cognee

### Bagaimana Cognee Berfungsi

Cognee menyediakan sistem memori yang canggih yang melangkaui penyimpanan kunci-nilai yang mudah:

```
┌──────────────────────────┐
│      30+ data sources    │
└───────────┬──────────────┘
            │
            ▼
┌──────────────────────────────────────────┐
│  Dynamically evolving memory layers      │
│                                          │
│  ┌────────────────────────────────────┐  │
│  │ Knowledge Graph in Graph Database  │  │
│  └────────────────────────────────────┘  │
│  ┌────────────────────────────────────┐  │
│  │ Embeddings in Vector Store         │  │
│  │   (e.g., Azure AI Search)          │  │
│  └────────────────────────────────────┘  │
└───────────┬──────────────────────────────┘
            │                      ▲   
            ▼                      │(optional)
┌────────────────┐           ┌────────────────┐
│     cognee     │(optional) │ Cognee Session │
│    retrievers  │──────────▶│     Cache      │
│                │           │    (Redis)     │
└───────┬────────┘           └────────────────┘
        ▲
        │
┌──────────────────────────┐
│          Agents          │
└──────────────────────────┘

```

### Komponen Utama:

1. **Graf Pengetahuan**: Menyimpan entiti, hubungan, dan sambungan semantik
2. **Vector Embeddings**: Membolehkan carian semantik merentasi semua maklumat yang disimpan
3. **Cache Sesi**: Mengekalkan konteks perbualan dalam dan antara sesi
4. **NodeSets**: Mengatur data ke dalam kategori logik untuk pengambilan yang disasarkan

### Jenis Memori dalam Tutorial Ini:

- **Memori Kekal**: Penyimpanan jangka panjang dalam graf pengetahuan
- **Memori Sesi**: Konteks perbualan sementara dalam cache Redis
- **Memori Semantik**: Carian kesamaan berasaskan vektor merentasi semua data


## 📦 Pasang Pakej Diperlukan

Pasang Cognee dengan sokongan Redis untuk pengurusan sesi:


In [ ]:
!pip install --quiet "cognee[redis]==0.4.0"

## 🔧 Memulakan Persekitaran dan Memuatkan Pustaka

Pastikan:
1. Redis sedang berjalan (contohnya, melalui Docker: `docker run -d -p 6379:6379 redis`)
2. Pembolehubah persekitaran telah ditetapkan sebelum mengimport modul cache
3. Jika perlu, mulakan semula kernel dan jalankan sel mengikut urutan

Sel berikut akan:
1. Memuatkan pembolehubah persekitaran dari `.env`
2. Mengkonfigurasi Cognee dengan tetapan LLM anda
3. Mengaktifkan cache untuk pengurusan sesi
4. Mengesahkan semua komponen disambungkan dengan betul


In [ ]:
import os
from pathlib import Path

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# cognee Configuration
os.environ["LLM_API_KEY"] = os.getenv("LLM_API_KEY", None)
os.environ["CACHING"] = os.getenv("CACHING", "true")


import cognee

print(f"Cognee version: {cognee.__version__}")
print(f"CACHING: {os.environ.get('CACHING')}")
print(f"LLM_API_KEY: {os.environ.get('LLM_API_KEY')}")

## 📁 Konfigurasi Direktori Penyimpanan

Cognee menggunakan dua direktori berasingan untuk operasinya:
- **Data Root**: Menyimpan dokumen yang dimasukkan dan data yang telah diproses
- **System Root**: Mengandungi pangkalan data graf pengetahuan dan metadata sistem

Kita akan mencipta direktori yang terasing untuk tutorial ini seperti berikut:


In [ ]:
DATA_ROOT = Path('.data_storage').resolve()
SYSTEM_ROOT = Path('.cognee_system').resolve()

DATA_ROOT.mkdir(parents=True, exist_ok=True)
SYSTEM_ROOT.mkdir(parents=True, exist_ok=True)

cognee.config.data_root_directory(str(DATA_ROOT))
cognee.config.system_root_directory(str(SYSTEM_ROOT))

print(f"Data root: {DATA_ROOT}")
print(f"System root: {SYSTEM_ROOT}")

## 🧹 Tetapkan Semula Keadaan Memori

Sebelum kita mula membina sistem memori kita, mari pastikan kita bermula dengan keadaan yang bersih.

> 💡 **Tip**: Anda boleh abaikan langkah ini jika anda ingin mengekalkan memori sedia ada daripada sesi sebelumnya apabila anda menggunakan buku nota ini nanti.


In [ ]:
await cognee.prune.prune_data()
await cognee.prune.prune_system(metadata=True)
print('Cleared previous Cognee state.')

## 📚 Bahagian 1: Membina Pangkalan Pengetahuan

### Sumber Data untuk Pembantu Pembangun Kita

Kita akan menggunakan tiga jenis data untuk mencipta pangkalan pengetahuan yang komprehensif:

1. **Profil Pembangun**: Kepakaran peribadi dan latar belakang teknikal
2. **Amalan Terbaik Python**: Zen of Python dengan panduan praktikal
3. **Perbualan Sejarah**: Sesi soal jawab terdahulu antara pembangun dan pembantu AI

Data yang pelbagai ini membolehkan agen kita untuk:
- Memahami konteks teknikal pengguna
- Menerapkan amalan terbaik dalam cadangan
- Belajar daripada interaksi berjaya yang lepas


In [ ]:
developer_intro = (
  "Hi, I'm an AI/Backend engineer. "
  "I build FastAPI services with Pydantic, heavy asyncio/aiohttp pipelines, "
  "and production testing via pytest-asyncio. "
  "I've shipped low-latency APIs on AWS, Azure, and GoogleCloud."
)

python_zen_principles = (
  """
    # The Zen of Python: Practical Guide

    ## Overview
    Use these principles as a checklist during design, coding, and reviews.

    ## Key Principles With Guidance

    ### 1. Beautiful is better than ugly
    Prefer descriptive names, clear structure, and consistent formatting.

    ### 2. Explicit is better than implicit
    Be clear about behavior, imports, and types.
    ```python
    from datetime import datetime, timedelta

    def get_future_date(days_ahead: int) -> datetime:
        return datetime.now() + timedelta(days=days_ahead)
    ```

    ### 3. Simple is better than complex
    Choose straightforward solutions first.

    ### 4. Complex is better than complicated
    When complexity is needed, organize it with clear abstractions.

    ### 5. Flat is better than nested
    Use early returns to reduce indentation.

    ## Modern Python Tie-ins
    - Type hints reinforce explicitness
    - Context managers enforce safe resource handling
    - Dataclasses improve readability for data containers

    ## Quick Review Checklist
    - Is it readable and explicit?
    - Is this the simplest working solution?
    - Are errors explicit and logged?
    - Are modules/namespaces used appropriately?
  """
)

human_agent_conversations = (
  """
  "conversations": [
      {
        "id": "conv_001",
        "timestamp": "2024-01-15T10:30:00Z",
        "topic": "async/await patterns",
        "user_query": "I'm building a web scraper that needs to handle thousands of URLs concurrently. What's the best way to structure this with asyncio?",
        "assistant_response": "Use asyncio with aiohttp, a semaphore to cap concurrency, TCPConnector for connection pooling, context managers for session lifecycle, and robust exception handling for failed requests.",
        "code_context": {
          "file": "scraper.py",
          "language": "python",
          "patterns_discussed": ["async/await", "context_managers", "semaphores", "aiohttp", "error_handling"]
        },
        "follow_up_questions": [
          "How do I add retry logic for failed requests?",
          "What's the best way to parse the scraped HTML content?"
        ]
      },
      {
        "id": "conv_002",
        "timestamp": "2024-01-16T14:20:00Z",
        "topic": "dataclass vs pydantic",
        "user_query": "When should I use dataclasses vs Pydantic models? I'm building an API and need to handle user input validation.",
        "assistant_response": "For API input/output, prefer Pydantic: it provides runtime validation, type coercion, JSON serialization, enums for roles, field constraints, and custom validators; integrates cleanly with FastAPI for automatic request validation and error reporting.",
        "code_context": {
          "file": "models.py",
          "language": "python",
          "patterns_discussed": ["pydantic", "dataclasses", "validation", "fastapi", "type_hints", "enums"]
        },
        "follow_up_questions": [
          "How do I handle nested validation with Pydantic?",
          "Can I use Pydantic with SQLAlchemy models?"
        ]
      },
      {
        "id": "conv_003",
        "timestamp": "2024-01-17T09:15:00Z",
        "topic": "testing patterns",
        "user_query": "I'm struggling with testing async code and database interactions. What's the best approach for pytest with async functions?",
        "assistant_response": "Recommended using pytest-asyncio, async fixtures, and an isolated test database or mocks to reliably test async functions and database interactions in FastAPI.",
        "code_context": {
          "file": "test_users.py",
          "language": "python",
          "patterns_discussed": ["pytest", "async_testing", "fixtures", "mocking", "database_testing", "fastapi_testing"]
        },
        "follow_up_questions": [
          "How do I test WebSocket connections?",
          "What's the best way to test database migrations?"
        ]
      },
      {
        "id": "conv_004",
        "timestamp": "2024-01-18T16:45:00Z",
        "topic": "performance optimization",
        "user_query": "My FastAPI app is getting slow with large datasets. How can I optimize database queries and response times?",
        "assistant_response": "Suggested optimizing database queries (indexes, pagination, selecting only needed columns), adding caching, streaming or chunked responses for large datasets, background tasks for heavy work, and monitoring to find bottlenecks.",
        "code_context": {
          "file": "optimizations.py",
          "language": "python",
          "patterns_discussed": ["performance_optimization", "caching", "database_optimization", "async_patterns", "monitoring"]
        },
        "follow_up_questions": [
          "How do I implement database connection pooling properly?",
          "What's the best way to handle memory usage with large datasets?"
        ]
      },
      {
        "id": "conv_005",
        "timestamp": "2024-01-19T11:30:00Z",
        "topic": "error handling and logging",
        "user_query": "I need to implement proper error handling and logging across my Python application. What's the best approach for production-ready error management?",
        "assistant_response": "Proposed centralized error handling with custom exceptions, structured logging, FastAPI middleware or decorators, and integration points for external monitoring/alerting tools.",
        "code_context": {
          "file": "error_handling.py",
          "language": "python",
          "patterns_discussed": ["error_handling", "logging", "exceptions", "middleware", "decorators", "fastapi"]
        },
        "follow_up_questions": [
          "How do I integrate this with external monitoring tools like Sentry?",
          "What's the best way to handle errors in background tasks?"
        ]
      }
    ],
    "metadata": {
      "total_conversations": 5,
      "date_range": "2024-01-15 to 2024-01-19",
      "topics_covered": [
        "async/await patterns",
        "dataclass vs pydantic",
        "testing patterns",
        "performance optimization",
        "error handling and logging"
      ],
      "code_patterns_discussed": [
        "asyncio", "aiohttp", "semaphores", "context_managers",
        "pydantic", "fastapi", "type_hints", "validation",
        "pytest", "async_testing", "fixtures", "mocking",
        "performance_optimization", "caching", "database_optimization",
        "error_handling", "logging", "exceptions", "middleware"
      ],
      "difficulty_levels": {
        "beginner": 1,
        "intermediate": 2,
        "advanced": 2
      }
    }
  """
)

## 🔄 Proses Data ke dalam Graf Pengetahuan

Sekarang kita akan menukar teks mentah kita menjadi memori yang berstruktur. Proses ini:

1. **Menambah data ke NodeSets**: Menyusun maklumat ke dalam kategori logik
   - `developer_data`: Profil pembangun dan perbualan
   - `principles_data`: Amalan terbaik dan panduan Python

2. **Menjalankan Cognify Pipeline**: Mengekstrak entiti, hubungan, dan mencipta embedding
   - Mengenal pasti konsep utama
   - Mewujudkan sambungan semantik antara maklumat yang berkaitan
   - Menjana embedding vektor

Ini mungkin mengambil masa beberapa saat kerana LLM memproses teks dan membina struktur graf:


In [ ]:
await cognee.add(developer_intro, node_set=["developer_data"])
await cognee.add(human_agent_conversations, node_set=["developer_data"])
await cognee.add(python_zen_principles, node_set=["principles_data"])

await cognee.cognify()

## 📊 Visualisasikan Graf Pengetahuan

Mari kita terokai struktur graf pengetahuan kita. Visualisasi ini menunjukkan:
- **Nod**: Entiti yang diekstrak daripada teks (konsep, teknologi, individu)
- **Tepi**: Hubungan dan sambungan antara entiti
- **Kumpulan**: Konsep berkaitan yang dikelompokkan berdasarkan kesamaan semantik

Buka fail HTML yang dijana dalam pelayar anda untuk meneroka graf secara interaktif:


In [ ]:
from cognee import visualize_graph
await visualize_graph('./visualization_1.html')

## 🧠 Perkayakan Memori dengan Memify

Fungsi `memify()` menganalisis graf pengetahuan dan menghasilkan peraturan pintar tentang data. Proses ini:
- Mengenal pasti corak dan amalan terbaik
- Mencipta panduan yang boleh diambil tindakan berdasarkan kandungan
- Menjalin hubungan antara pelbagai bidang pengetahuan

Peraturan ini membantu agen membuat keputusan yang lebih bermaklumat semasa menjawab soalan. Menangkap visualisasi kedua membantu anda membandingkan bagaimana graf menjadi lebih padat setelah diperkayakan.


In [ ]:
await cognee.memify()

await visualize_graph('./visualization_2.html')

## 🔍 Bahagian 2: Pengambilan Memori Pintar

### Demonstrasi 1: Integrasi Pengetahuan Merentas Dokumen

Sekarang bahawa graf pengetahuan kita telah dibina, mari kita uji bagaimana Cognee menggabungkan maklumat dari pelbagai sumber untuk menjawab soalan yang kompleks.

Pertanyaan pertama menunjukkan:
- **Pemahaman semantik**: Mencari konsep yang relevan walaupun tidak disebut secara eksplisit
- **Rujukan silang**: Menggabungkan profil pembangun dengan prinsip Python
- **Penaakulan kontekstual**: Menerapkan amalan terbaik kepada pelaksanaan tertentu

### Demonstrasi 2: Carian Tapis dengan NodeSets

Pertanyaan kedua menunjukkan cara menyasarkan subset tertentu dalam graf pengetahuan:
- Menggunakan parameter `node_name` untuk mencari hanya dalam `principles_data`
- Memberikan jawapan yang fokus dari domain pengetahuan tertentu
- Berguna apabila anda memerlukan maklumat khusus domain


In [ ]:
# demonstrate cross-document knowledge retrieval from multiple data sources
from cognee.modules.search.types import SearchType

results = await cognee.search(
    query_text="How does my AsyncWebScraper implementation align with Python's design principles?",
    query_type=SearchType.GRAPH_COMPLETION,
)
print("Python Pattern Analysis:", results)

# demonstrate filtered search using NodeSet to query only specific subsets of memory
from cognee.modules.engine.models.node_set import NodeSet
results = await cognee.search(
    query_text="How should variables be named?",
    query_type=SearchType.GRAPH_COMPLETION,
    node_type=NodeSet,
    node_name=["principles_data"],
)
print("Filtered search result:", results)

## 🔐 Bahagian 3: Tetapan Pengurusan Sesi

### Mengaktifkan Memori Perbualan

Pengurusan sesi adalah penting untuk mengekalkan konteks sepanjang pelbagai interaksi. Di sini kita akan:

1. **Inisialisasi Konteks Pengguna**: Cipta atau dapatkan profil pengguna untuk penjejakan sesi
2. **Konfigurasi Enjin Cache**: Sambungkan ke Redis untuk menyimpan sejarah perbualan
3. **Aktifkan Pembolehubah Sesi**: Tetapkan pembolehubah konteks yang kekal merentasi pertanyaan

> ⚠️ **Penting**: Ini memerlukan Redis sedang berjalan dan `CACHING=true` dalam persekitaran anda


In [ ]:
from cognee.modules.users.methods import get_default_user
from cognee.context_global_variables import set_session_user_context_variable 
from cognee.infrastructure.databases.cache import get_cache_engine

user = await get_default_user()
await set_session_user_context_variable(user)
print(f"Using user id: {getattr(user, 'id', 'unknown')}")

cache_engine = get_cache_engine()
if cache_engine is None:
    raise RuntimeError('Cache engine is not available. Double-check your cache configuration.')
print('Session cache is ready.')


## 🛠️ Fungsi Pembantu: Lihat Sejarah Sesi

Fungsi utiliti ini membolehkan kita memeriksa sejarah perbualan yang disimpan dalam Redis. Ia berguna untuk:
- Menyelesaikan masalah pengurusan sesi
- Memastikan perbualan disimpan dalam cache
- Memahami konteks yang tersedia untuk ejen


In [ ]:
async def show_history(session_id: str) -> None:
    # Let's check the cache directly
    cache_engine = get_cache_engine()
    if cache_engine:
        # Try to get history directly from cache
        user_id = str(user.id) if hasattr(user, 'id') else None
        if user_id:
            history_entries = await cache_engine.get_latest_qa(user_id, session_id, last_n=10)
            print(f"\nDirect cache query for user_id={user_id}, session_id={session_id}:")
            print(f"Found {len(history_entries)} entries")
            if history_entries:
                for i, entry in enumerate(history_entries, 1):
                    print(f"\nEntry {i}:")
                    print(f"  Question: {entry.get('question', 'N/A')[:100]}...")
                    print(f"  Answer: {entry.get('answer', 'N/A')[:100]}...")
        else:
            print("No user_id available")


## Sesi 1: Makmal Sokongan Async — Soalan Pertama

Mulakan sesi `async-support-lab` dengan bertanya tentang corak asyncio yang mesra telemetri untuk pengikis web berskala besar. Graf sudah mengetahui tentang asyncio, aiohttp, dan amalan pemantauan, jadi responsnya harus mencerminkan perbualan sebelumnya sambil menyesuaikan jawapan dengan pertanyaan baru.


In [ ]:
session_1 = "async-support-lab"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="I'm building a web scraper that hits thousands of URLs concurrently. What's a reliable asyncio pattern with telemetry?",
    session_id=session_1
)

## Periksa Memori Sesi 1 Selepas Pertukaran Pertama

Menjalankan `show_history(session_1)` sejurus selepas soalan awal mengesahkan bahawa Cognee telah menulis kedua-dua prompt dan jawapan ke dalam Redis. Anda sepatutnya melihat satu entri dengan panduan keserentakan.


In [ ]:
await show_history(session_1)

## Sesi 1: Susulan mengenai Model Data

Seterusnya kita tanya, "Bila saya patut memilih dataclasses berbanding Pydantic?" menggunakan ID sesi yang sama. Cognee sepatutnya menggabungkan prinsip Python bersama perbincangan FastAPI sebelum ini untuk memberikan nasihat yang lebih terperinci—menunjukkan bahawa konteks dibawa ke hadapan dalam sesi yang dinamakan.


In [ ]:
result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="When should I pick dataclasses versus Pydantic for this work?",
    session_id=session_1
)

## Sahkan Sejarah Sesi 1 Mengandungi Kedua-dua Giliran

Panggilan `show_history(session_1)` yang lain sepatutnya menunjukkan dua entri Soal & Jawab. Ini sepadan dengan langkah "ulang ingatan" dalam makmal Mem0 dan membuktikan bahawa giliran tambahan melanjutkan transkrip yang sama.


In [ ]:
await show_history(session_1)

## Sesi 2: Benang Semakan Reka Bentuk — Sesi Baru

Untuk menunjukkan pengasingan antara benang, kami memulakan `design-review-session` dan meminta panduan log untuk semakan insiden. Walaupun asas pangkalan pengetahuan adalah sama, id sesi baru memastikan transkrip kekal berasingan.


In [ ]:
session_2 = "design-review-session"

result = await cognee.search(
    query_type=SearchType.GRAPH_COMPLETION,
    query_text="We're drafting logging guidance for incident reviews. Capture the key principles please.",
    session_id=session_2
)

## Ulasan Sesi 2 Sejarah

`show_history(session_2)` hanya perlu menyenaraikan pasangan prompt/respons ulasan reka bentuk. Bandingkan dengan Sesi 1 untuk menonjolkan bagaimana Cognee mengekalkan transkrip yang berasingan sambil menggunakan semula graf pengetahuan yang dikongsi.


In [ ]:
await show_history(session_2)

## Ringkasan

Tahniah! Anda baru sahaja memberikan pembantu pengekodan anda lapisan memori jangka panjang sebenar yang dikuasakan oleh Cognee.

Dalam tutorial ini, anda telah mengambil kandungan pembangun mentah (kod, dokumen, sembang) dan mengubahnya menjadi graf + memori vektor yang boleh dicari, dianalisis, dan diperbaiki secara berterusan oleh agen anda.

Apa Yang Anda Pelajari

1. **Daripada teks mentah kepada memori AI**: Bagaimana Cognee memproses data tidak berstruktur dan mengubahnya menjadi memori pintar yang boleh dicari menggunakan gabungan seni bina vektor + graf pengetahuan.

2. **Pengayaan graf dengan memify**: Bagaimana untuk melangkaui penciptaan graf asas dan menggunakan memify untuk menambah fakta terbitan dan hubungan yang lebih kaya di atas graf sedia ada anda.

3. **Pelbagai strategi carian**: Bagaimana untuk membuat pertanyaan memori dengan pelbagai jenis carian (Q&A yang sedar graf, pelengkapan gaya RAG, wawasan, potongan mentah, carian kod, dll.) bergantung pada keperluan agen anda.

4. **Penerokaan visual**: Bagaimana untuk memeriksa dan menyahpepijat apa yang dibina oleh Cognee menggunakan visualisasi graf dan UI Cognee, supaya anda benar-benar dapat melihat bagaimana pengetahuan disusun.

5. **Memori sedar sesi**: Bagaimana untuk menggabungkan konteks setiap sesi dengan memori semantik berterusan supaya agen dapat mengingati merentasi sesi tanpa membocorkan maklumat antara pengguna.


## Poin Penting
1. Memori sebagai Graf Pengetahuan yang disokong oleh Embeddings

    - **Pemahaman terstruktur**: Cognee menggabungkan stor vektor dan stor graf supaya data anda boleh dicari berdasarkan makna dan dihubungkan melalui hubungan. Cognee menggunakan pangkalan data berasaskan fail secara lalai (LanceDB untuk stor vektor, Kuzu untuk pangkalan data graf).

    - **Pengambilan maklumat berasaskan hubungan**: Jawapan boleh diasaskan bukan sahaja pada "teks yang serupa," tetapi juga pada bagaimana entiti saling berkaitan.

    - **Memori yang hidup**: Lapisan memori berkembang, bertambah, dan kekal boleh dicari sebagai satu graf yang bersambung.

2. Mod Carian & Penaakulan
    - **Pengambilan hibrid**: Carian menggabungkan kesamaan vektor, struktur graf, dan penaakulan LLM, dari pencarian potongan mentah hingga soal jawab yang sedar graf.

    - **Sesuaikan mod dengan tugas**: Gunakan mod gaya pelengkap apabila anda mahukan jawapan dalam bahasa semula jadi, dan mod potongan/ringkasan/graf apabila agen anda memerlukan konteks mentah atau untuk mendorong penaakulan sendiri.

3. Agen yang Diperibadikan dan Sedar Sesi
    - **Konteks sesi + memori jangka panjang**: Cognee memisahkan konteks "benang" jangka pendek daripada memori jangka panjang di peringkat pengguna atau organisasi.

## Aplikasi Dunia Sebenar

1. **Agen AI Vertikal**

    Gunakan corak dari buku nota ini untuk menggerakkan pembantu pintar domain yang duduk di atas Cognee sebagai teras pengambilan dan penaakulan mereka:

- **Pembantu pembangun**: Semakan kod, analisis insiden, dan pembantu seni bina yang menelusuri kod, API, dokumen reka bentuk, dan tiket sebagai satu graf memori.

- **Pembantu berhadapan pelanggan**: Agen sokongan atau kejayaan yang menarik dari dokumen produk, FAQ, nota CRM, dan tiket lalu dengan pengambilan sedar graf dan jawapan yang disokong.

- **Pembantu pakar dalaman**: Pembantu dasar, undang-undang, atau keselamatan yang membuat penaakulan berdasarkan peraturan, panduan, dan keputusan sejarah yang saling berkaitan, bukannya PDF yang terasing.

    Cognee secara eksplisit diposisikan sebagai memori yang berterusan dan tepat untuk agen AI, menyediakan graf pengetahuan yang hidup yang menggantikan kombinasi ad-hoc stor vektor dan kod graf tersuai.

2. **Menyatukan Silo Data ke dalam Satu Memori**

    Pendekatan yang sama juga membantu anda membina lapisan memori bersatu merentasi sumber yang tersebar:

- **Dari silo ke satu graf**: Serap data terstruktur (contohnya, pangkalan data) dan tidak terstruktur (contohnya, dokumen, sembang) ke dalam satu graf yang disokong oleh embeddings, bukannya indeks berasingan untuk setiap sistem.

- **Penaakulan silang sumber dengan sitasi**: Jalankan penaakulan berbilang langkah ke atas semuanya—“gabungkan” log, metrik, dan dokumen melalui graf—dan tetap kembalikan jawapan yang diasaskan dengan bukti.

- **Hab pengetahuan**: Untuk domain seperti perbankan atau pendidikan, Cognee sudah digunakan untuk menyatukan PDF, sistem dalaman, dan data aplikasi ke dalam satu graf pengetahuan dengan vektor supaya agen boleh menjawab soalan dengan konteks yang tepat dan disokong.

## Langkah Seterusnya

Anda telah melaksanakan gelung memori teras. Berikut adalah lanjutan semula jadi yang boleh anda cuba sendiri (lihat [dokumentasi Cognee](https://docs.cognee.ai/) untuk butiran):

1. **Bereksperimen dengan kesedaran temporal**: Hidupkan temporal cognify untuk mengekstrak peristiwa dan cap waktu dari teks.

2. **Perkenalkan penaakulan berasaskan ontologi**: Tentukan ontologi OWL untuk domain anda. Gunakan sokongan ontologi Cognee supaya entiti dan hubungan yang diekstrak diasaskan dalam skema tersebut, meningkatkan kualiti graf dan jawapan khusus domain.

3. **Tambah gelung maklum balas**: Biarkan Cognee menyesuaikan berat tepi graf berdasarkan maklum balas pengguna sebenar, supaya pengambilan maklumat bertambah baik dari masa ke masa dan tidak statik.

4. **Laraskan untuk personalisasi & tingkah laku sesi**: Gunakan ID pengguna, penyewa, dan set data untuk memberikan setiap individu atau pasukan pandangan mereka sendiri terhadap enjin memori yang dikongsi.

5. **Skalakan kepada agen yang lebih kompleks**: Sambungkan Cognee ke rangka kerja agen untuk membina sistem multi-agen yang semuanya berkongsi lapisan memori yang sama. *Microsoft Agent Framework x plugin Cognee akan datang tidak lama lagi.*


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan perkhidmatan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Walaupun kami berusaha untuk ketepatan, sila ambil perhatian bahawa terjemahan automatik mungkin mengandungi kesilapan atau ketidaktepatan. Dokumen asal dalam bahasa asalnya harus dianggap sebagai sumber yang berwibawa. Untuk maklumat kritikal, terjemahan manusia profesional adalah disyorkan. Kami tidak bertanggungjawab atas sebarang salah faham atau salah tafsir yang timbul daripada penggunaan terjemahan ini.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
